# Proposition d'algorithme pour le choix des PFR

## Principe 

Un principe itératif:
1. on regarde pour chacune son ou ses projets préférés, puis on détermine tous les groupes valides que l'on peut faire à partir de là.
2. si aucune solution n'est retournée, on baisse d'un point les notes maximales (on passe toutes les notes 50 à 49 par exemple) et on recommence au point 1.
3. si au moins une solution est retournée, c'est fini. Il reste à choisir la solution en bonne intelligence...

De cette manière on cherche à satisfaire chacun au mieux, si ce n'est pas possible on *baisse un peu la barre* pour tout le monde et on recommence. Je trouve que c'est mieux que de chercher à maximiser une fonction globale qui pourrait satisfaire beaucoup la majorité et laisser quelques très déçus.

Par ailleurs, cet algorithme propose l'ensemble des (meilleures) possibilités, il n'y a pas d'aléatoire. L'implémentation est (assez) simple, mais un peu longue à tourner.

Principe de la nouvelle implémentation :  
- on part d'une matrice M de booleans (Mij == True <=> le projet_j fait partie des projets préférés de la personne_i) ;
- on peut tester si la matrice représente une configuration acceptable (i.e. si chaque personne n'a qu'un seul projet affecté et que les groupes sont tous de 4 personnes sauf un groupe de 3 personnes)
- si la matrice n'est pas acceptable, alors on tente de la rendre acceptable en la transformant (on transforme un ou plusieurs True en False)
- on continue en testant toutes les possibilités jusqu'à obtenir soit une matrice acceptable, soit une matrice non acceptable (e.g. avec une personne affectée à 0 projets).

L'ensemble des possibilités étant très large, on tente de faire des transformation intelligentes et de détecter au plus tôt les configurations qui ne peuvent aboutir à une matrice acceptable, quelque soit les futures transformations appliquées.

## Algorithme et exemple

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

class DataError(Exception):
    pass

Données exemples, aléatoires.

In [2]:
np.random.seed(88)
nb_projects = 4
group_sizes = [3, 4]
nb_persons = 4 * nb_projects
notation_types = [[50, 49, 1], [21, 20, 20, 19], [50, 25, 25]]
def random_row():
    notation = notation_types[np.random.randint(len(notation_types))]
    row = np.zeros(nb_projects)
    for n, i in zip(notation, np.random.permutation(nb_projects)):
        row[i] = n
    return row
dat = pd.DataFrame(data=[random_row() for i in range(nb_persons)],
                   columns=['Projet_{}'.format(i) for i in range(nb_projects)],
                   index=['Person_{}'.format(i) for i in range(nb_persons)])

In [3]:
dat

,Projet_0,Projet_1,Projet_2,Projet_3
Person_0,0,1,50,49
Person_1,50,0,1,49
Person_2,1,0,49,50
Person_3,50,1,0,49
Person_4,50,49,1,0
Person_5,50,25,0,25
Person_6,21,20,20,19
Person_7,1,50,0,49
Person_8,49,50,0,1
Person_9,50,0,49,1


Fonctions utiles.

In [4]:
def make_M(df, cut):
    '''Transform the initial dataframe into a matrix of bolean M such as:
    Mij = True iff project_j is one of the person_i's preferred project
    Parameters:
    -----------
    df : the initial dataframe
    cut : the value at which a project is considered to be a 'preferred' project
    Returns:
    --------
    A n x p boolean Numpy array.
    '''
    val = df.values.copy()
    val[val > cut] = cut
    max_lines = val.max(axis=1)
    M = (val.T == max_lines).T
    return M

In [5]:
def clean_and_check_M(M):
    '''Remove (in-place) some impossible choices (the more and the faster we can find them the better).
    Then test for (in)validity
    Current cleaning ideas:
    (i) removing (setting the column to False) projects with strictly less than 3 people on it
    (ii) removing people from projects who are full (i.e. already 4 people sure on it) ;
         may need several passes
    Current checking for invalidity ideas:
    (i) there exists i such that sum_i Mij == 0
    (ii) there is a project with more than 4 people, sure, on it
    Current checking for validity ideas:
    (i) sum_j Mij == 1 for all i
    (ii) [sum_i Mij for i in 1..n] contains only 0s, 4s and at most one 3.
    Parameters:
    -----------
    M : numpy boolean array to be cleaned
    Returns:
    --------
    -1 if invalid
    +1 if valid
     0 if we do not know and must continue our search
    '''
    # Cleaning.. 
    sum_cols = M.sum(axis=0)
    sum_lines = M.sum(axis=1)
    # .. (i)
    M[:, sum_cols < 3] = False
    # .. (ii)
    flag = True
    while flag:
        pers_sure = sum_lines == 1
        full_projs = M[pers_sure].sum(axis=0) >= 4
        sub_M = M[np.ix_(np.invert(pers_sure), full_projs)]
        flag = np.any(sub_M) # if there is something to change
        #print("flag is {}...".format(flag))
        if flag:
            M[np.ix_(np.invert(pers_sure), full_projs)] = False
            
    # Checking... 
    sum_lines = M.sum(axis=1)
    sum_cols = M.sum(axis=0)
    pers_sure = sum_lines == 1
    # .. if invalid
    # (i) (ii)
    if np.any(sum_lines == 0) or \
    np.any(M[pers_sure].sum(axis=0) > 4):
        return -1
    
    # .. if valid
    if np.all(pers_sure) and \
    (not [1 for s in sum_cols if s not in [0, 4, 3]]) and \
    (np.sum(sum_cols == 3) <= 1):
        return 1
    
    # If we do not know...
    return 0

In [6]:
def next_M(M):
    '''returns a list of copies of the matrix M with each correspond to a step towards
    a solution
    Best ideas so far : 
    (i) we pick the person with the fewest number of preferred projects (k > 1) and
        create k different possible matrices
    '''
    sum_lines = M.sum(axis=1)
    sum_lines[sum_lines == 1] = 32768 # arbitrary large value, must be larger than the number of projects
    pers = np.argmin(sum_lines)
    for j, proj in enumerate(M[pers]):
        if proj:
            new_M = M.copy()
            new_M[pers] = 0
            new_M[pers, j] = True
            yield new_M
    #Can we assure that the cleaned version will yield different matrices ??? 
    # -> In that case, yes since the cleaned matrices will either be different or invalid
    

Fonction principale de l'algorithme, qui explore l'arbre des possibilités.

In [7]:
def make_groups(M):
    '''Returns a list of acceptable projects
    
    Parameters:
    -----------
    M : an nxp Numpy boolean array representing preferred projects per persons
    Returns:
    --------
    a list of valid numpy arrays 
    '''
    sols = []
    Ms = [M]
    for m in Ms:
        #print('We check \n {}'.format(m))
        check = clean_and_check_M(m)
        if check == 1: # if valid
            #print('{} is valid : horray !'.format(m))
            sols.append(m)
        elif check == 0: # if we don't know, we investigate further
            #print('it may be valid... we investigate')
            for n_m in next_M(m):
                #print('we add to the list \n {}'.format(n_m))
                Ms.append(n_m)
        else: # if invalid
            #print('it is not valid.')
            pass
    return sols

In [8]:
m_test = np.array([[True,False,False, False], [True,False,False, False], [True,False,False, False],
                   [True,True,True, False], [False,False,True, True],[False,False,True, False],
                   [False,False,True, False]])
print(m_test)

[[ True False False False]
 [ True False False False]
 [ True False False False]
 [ True  True  True False]
 [False False  True  True]
 [False False  True False]
 [False False  True False]]


In [9]:
make_groups(m_test)

[array([[ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False]], dtype=bool),
 array([[ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False]], dtype=bool)]

In [10]:
m_test.max(axis=1)

array([ True,  True,  True,  True,  True,  True,  True], dtype=bool)

Fonction utile qui cappe les valeurs à une certaine valeur :

In [11]:
def haircut(dat, n):
    ''' Caps all numbers at n'''
    dat_cap = dat.applymap(lambda x: min(n, x))
    return dat_cap

L'algorithme complet :

In [12]:
def algo(dat, cap=50):
    '''Main algorithm (recursive)'''
    # If the data is correct, this should not happen
    # put it there to prevent infinite loop
    if cap == 1:
        raise DataError('Bad data, no solution is possible')
    print("Run for cap={}...".format(cap))
    m = make_M(dat, cap)
    sol = make_groups(m)    
    if not sol:
        return algo(dat, cap=cap-1)
    return sol   

Test avec les données aléatoires :

In [13]:
make_M(dat,50)

array([[False, False,  True, False],
       [ True, False, False, False],
       [False, False, False,  True],
       [ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       [False,  True, False, False],
       [False,  True, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       [False, False,  True, False],
       [False, False, False,  True],
       [False, False, False,  True],
       [False,  True, False, False]], dtype=bool)

In [14]:
algo(dat)

Run for cap=50...
Run for cap=49...


[array([[False, False,  True, False],
        [False, False, False,  True],
        [False, False,  True, False],
        [False, False, False,  True],
        [False,  True, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [False,  True, False, False],
        [False,  True, False, False],
        [False, False,  True, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [False, False,  True, False],
        [False, False, False,  True],
        [False, False, False,  True],
        [False,  True, False, False]], dtype=bool),
 array([[False, False,  True, False],
        [False, False, False,  True],
        [False, False,  True, False],
        [False, False, False,  True],
        [False,  True, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [False, False, False,  True],
        [False,  True, False, False],
        [False, False,  True, False]

---

### Avec nos données :

Import et nettoyage des données.

In [15]:
import requests
from io import StringIO
r = requests.get('https://docs.google.com/spreadsheets/d/1hUWvO8wyEJL-_SkhgpdrwdYiDL7XQdrbTkcp21py8Lw/export?format=csv&id=1hUWvO8wyEJL-_SkhgpdrwdYiDL7XQdrbTkcp21py8Lw&gid=0')

In [40]:
dat_bgd = pd.read_csv(StringIO(r.text), skiprows=1, index_col='Nom', encoding='utf8')
dat_bgd = dat_bgd.loc[:, 'Clustaar':'Plume Labs']
dat_bgd = dat_bgd.fillna(0)
dat_bgd = dat_bgd.loc[[i for i in dat_bgd.index if isinstance(i, str)], :]

In [41]:
dat_bgd

,Clustaar,Kernix Lab,Total,BNP,Amaury,STIF,SFR 1,SFR 2,Alstom,SACEM,FootBar,DCBrain,IPSEN,GrDF,Plume Labs
Nom,,,,,,,,,,,,,,,
Julien Cloud,0,0,0,0,0,0,0,0,1,0,0,49,0,0,50
William BENHAIM,50,0,0,0,0,0,0,0,0,0,0,26,0,0,24
Guillaume MOHR,1,0,0,0,0,0,0,0,0,0,0,50,0,0,49
Jade Lu Dac,0,0,0,40,0,20,0,0,0,20,0,0,0,20,0
Olivier Large,0,0,0,0,0,0,0,0,1,0,0,50,0,0,49
Kim Pellegrin,0,40,0,0,0,0,0,0,0,10,0,50,0,0,0
Malik OUSSALAH,0,25,0,0,0,0,0,0,0,0,0,0,50,0,25
Catherine Verdier,12,8,10,0,0,12,0,0,2,0,12,12,10,10,12
Cyril Gilbert,0,0,50,5,0,20,0,0,0,0,25,0,0,0,0


In [42]:
dat_bgd.shape

(30, 15)

In [43]:
sorted([n[1].upper() for n in dat_bgd.index.str.split(' ')])

['AKIL',
 'ALILI',
 'BENHAIM',
 'CARBONNE',
 'CHOUROU',
 'CLOUD',
 'CORBEL',
 'FDNZ',
 'FIORI',
 'FIRMIN',
 'GALICHER',
 'GILBERT',
 'GUEZ',
 'GUICHARD',
 'HO',
 'KUBRYK',
 'LAAROUSSI',
 'LARGE',
 'LAURENT',
 'LU',
 'MOHR',
 'OUSSALAH',
 'PELLEGRIN',
 'PIOCHAUD',
 'SALEM',
 'SIMEONIDIS',
 'SOGLO',
 'TODOROV',
 'VERDIER',
 'WEISSERT']

Il manque Cynthia (il faut qu'il y ait 31 personnes sinon l'algo ne trouvera rien !):

In [47]:
cv = pd.Series(name='Cynthia VARIEUX', index=dat_bgd.columns, data=[100//15] * 15)
dat_bgd = dat_bgd.append(cv)

Exécution de l'algorithme (attention, ça peut être long).

In [48]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
sol = algo(dat_bgd)
t1 = datetime.datetime.now()
print('enlapsed time = {}'.format(t1-t0))